In [1]:
import cellshape_cloud as cs_cloud
import torch
from torch.utils.data import DataLoader
from chamferdist import ChamferDistance

In [ ]:
PATH_TO_DATASET = "/home/mvries/Documents/Datasets/OPM/SingleCellFromNathan_17122021/Plate3/stacked_pointcloud"
batch_size = 20
learning_rate = 0.00001
num_features = 128
encoder_type = "dgcnn"
decoder_type = "foldingnetbasic"

model = cs_cloud.GraphAutoEncoder(
    num_features=num_features,
    k=k,
    encoder_type=encoder_type,
    decoder_type=decoder_type,
)

dataset = cs_cloud.PointCloudDataset(PATH_TO_DATASET)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

criterion = ChamferDistance()

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate * 16 / batch_size,
    betas=(0.9, 0.999),
    weight_decay=1e-6,
)

cs_cloud.train(model, dataloader, 1, criterion, optimizer, "./")